# カメラ画像処理について

## カメラ画像処理について

### この章について

この章ではカメラのしくみやRAW画像からはじめて、カメラ画像処理やRAW現像ソフトの中でどのような処理が行われているのかを説明します。

この章の内容については技術書典6で配布した書籍「ゼロから作るRAW現像」を参照下さい。

### この章のまとめ

カメラ内部の仕組みとRAW現像処理・カメラ画像処理を、画像処理の観点から説明しました。

### 次の章

次は[RAW画像の準備](http://bit.ly/raw_chap_3)を行います。